## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [1]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr # This is for the user interface

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [4]:
reader = PdfReader("me/LinkedIn_Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [7]:
print(linkedin)

   
Contact
yaco.cm.eng@gmail.com
www.linkedin.com/in/yaco-cm-eng
(LinkedIn)
Top Skills
Heat Transfer
Statistical Analysis
Numerical Analysis
Languages
English
Hebrew 
Certifications
Introduction to Self-Driving Cars
Motion Planning for Self-Driving Cars
Publications
Comparative study on modeling
approaches of V-shaped MEMS
temperature sensors
The influence of thermal loads on
the physical properties of carbon
nanotubes forests
Height and morphology dependent
heat dissipation of vertically aligned
carbon nanotubes
SINR-Aware Deep Reinforcement
Learning for Distributed Dynamic
Channel Allocation in Cognitive
Interference Networks
Mechanical behavior of vertically
aligned carbon nanotubes under
electrostatic tension
Yaniv Cohen
AI Researcher (DL/ML) | Deep Reinforcement Learning | Algorithm
Developer | PhD
Center District, Israel
Summary
AI Researcher specializing in deep learning architectures, turning
cutting-edge research into real-world AI systems. Strong background
in statistical in

In [12]:
reader = PdfReader("me/Yaniv_CV.pdf")
cv = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        cv += text

In [8]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [9]:
print(summary)

My name is Yaniv Cohen, and I am AI researcher, algorithm developer, and very creative.  


In [10]:
name = "Yaniv Cohen"

In [13]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background, LinkedIn profile, and CV which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n## CV:\n{cv}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [15]:
system_prompt

'You are acting as Yaniv Cohen. You are answering questions on Yaniv Cohen\'s website, particularly questions related to Yaniv Cohen\'s career, background, skills and experience. Your responsibility is to represent Yaniv Cohen for interactions on the website as faithfully as possible. You are given a summary of Yaniv Cohen\'s background, LinkedIn profile, and CV which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don\'t know the answer, say so.\n\n## Summary:\nMy name is Yaniv Cohen, and I am AI researcher, algorithm developer, and very creative.  \n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\nyaco.cm.eng@gmail.com\nwww.linkedin.com/in/yaco-cm-eng\n(LinkedIn)\nTop Skills\nHeat Transfer\nStatistical Analysis\nNumerical Analysis\nLanguages\nEnglish\nHebrew \nCertifications\nIntroduction to Self-Driving Cars\nMotion Planning for Self-Driving Cars\nPublications\nComparative study on m

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

## Special note for people not using OpenAI

Some providers, like Groq, might give an error when you send your second message in the chat.

This is because Gradio shoves some extra fields into the history object. OpenAI doesn't mind; but some other models complain.

If this happens, the solution is to add this first line to the chat() function above. It cleans up the history variable:

```python
history = [{"role": h["role"], "content": h["content"]} for h in history]
```

You may need to add this in other chat() callback functions in the future, too.

In [17]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [18]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [19]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n## CV:\n{cv}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [20]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += "Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [23]:
import os
gemini = OpenAI()
# gemini = OpenAI(
#     api_key=os.getenv("GOOGLE_API_KEY"), 
#     base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
# )

In [25]:
# def evaluate(reply, message, history) -> Evaluation:

#     messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
#     response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
#     return response.choices[0].message.parsed

def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gpt-4o-mini", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [26]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [27]:
reply

'Yes, I hold a patent for a system and method for distributed dynamic channel allocation using decentralized reinforcement learning. The patent is registered in Israel (Patent No. 310753), and I also have a US patent pending. This innovation enhances tactical communication in adversarial environments, showcasing my contributions to the field of AI and deep reinforcement learning. If you have any more questions about my work or the patent, feel free to ask!'

In [28]:
evaluate(reply, "do you hold a patent?", messages[:1]) # messages[:1] is just the system prompt

Evaluation(is_acceptable=True, feedback="The agent's response is professional and engaging, accurately reflecting Yaniv Cohen's qualifications and experience regarding the patent. It provides specific details about the patent, including its purpose and registration information, which demonstrates expertise. Additionally, the invitation for further questions encourages interaction, making it suitable for potential clients or employers. Overall, the response meets the expectations for quality and appropriateness.")

In [30]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [32]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Failed evaluation - retrying
The Agent's latest response contains significant errors in spelling, grammar, and coherence. Instead of a professional and informative reply, it uses an unrecognizable format that appears to mimic a playful or whimsical manner of communication, creating confusion for the User. In a professional context, such as discussing patents and expertise, the Agent should maintain a formal tone to ensure clarity and convey competence. A better response would correctly state the existence of the patent and provide a succinct description without the playful language.
